In [132]:
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import timedelta  


## Load sites data, and clean data

In [276]:
los_altos = pd.read_csv('../data/raw_data/los_altos/los_altos_permits.csv', sep='\t')

In [277]:
# make all column names a single lower case word
los_altos.rename({'Issued Date': 'issued', 
                  'Applied Date': 'applied', 
                  'Finaled Date': 'final', 
                  'Site APN': 'apn', 
                  "Site Address": 'address',
                  'Permit Type': 'type',
                  'Permit Subtype': 'subtype',
                  'Permit Number': 'id'}, axis=1, inplace=True)
los_altos.columns = map(str.lower, los_altos.columns)

In [278]:
los_altos.head()

,id,type,subtype,status,description,address,apn,applied,issued,final
0,1995-627536,DEMOLITION,NaN,ISSUED,,382 FIRST ST,16741005,3/22/1995,3/22/1995,NaN
1,1995-627599,DEMOLITION,NaN,ISSUED,PERMIT.,300 SPAGNOLI CT,16724033,4/4/1995,1/1/1980,NaN
2,1995-627667,DEMOLITION,NaN,FINALED,Int Remodel -111 & 113,300 SECOND ST,16740043,4/17/1995,4/17/1995,4/24/1994
3,1995-627692,DEMOLITION,NaN,FINALED,SFD,170 LOS ALTOS AV,16735074,4/20/1995,9/19/1995,4/11/1996
4,1995-627762,DEMOLITION,NaN,FINALED,Demo,422 CHERRY AV,16727051,5/2/1995,5/2/1995,6/5/1995


In [279]:
los_altos.type.value_counts()

DEMOLITION      2128
PHOTOVOLTAIC    2002
SFD              630
MULTI FAMILY     232
ADU               84
MIXED USE         17
Name: type, dtype: int64

Remove non-housing

In [280]:
los_altos = los_altos.query('type != "PHOTOVOLTAIC"')

### Correct data type

In [281]:
date_cols = ['applied', 'issued', 'final']
los_altos[date_cols] = los_altos[date_cols].apply(pd.to_datetime)

In [282]:
los_altos.apn = los_altos.apn.str.replace(r"[\-a-zA-Z|.+,;:/]", '', regex=True)
los_altos.apn = pd.to_numeric(los_altos.apn)

In [283]:
demolitions = los_altos.query('(issued >= "2013" and type == "DEMOLITION")')

In [284]:
homes = los_altos.query('issued >= 2015 and type != "DEMOLITION"')

### Flag rebuild

In [285]:
homes['demolished'] = 0

In [286]:
def find_rebuild(demolition, homes):
    """For a demolition permit, find the soonest rebuild. Print delay."""
    apn, address, pid, date = demolition.apn, demolition.address, demolition.id, demolition.issued
    by_date = date + timedelta(days=365*2)
    matches = homes.query('(apn == @apn or address == @address) and id != @pid and type != "DEMOLITION" and applied < @by_date').copy()
    matches.sort_values('applied', inplace=True)
    if len(matches.index):
        return matches.index[0]
    return False

In [287]:
homes.shape

(358, 11)

In [288]:
for i, demolition in demolitions.iterrows():
    idx = find_rebuild(demolition, homes)
    if idx:
        homes.demolished[idx] = 1

In [289]:
homes[homes.type == 'SFD'].demolished.sum() / homes.demolished.sum()

0.9421487603305785

In [291]:
manual = pd.read_csv('../data/raw_data/los_altos_manual.csv')

In [292]:
manual.rename({'pid': 'id'}, axis=1, inplace=True)

In [301]:
results = homes.merge(manual, how='left', left_on=['id', 'apn', 'address'], right_on=['id', 'apn', 'address'])

In [302]:
results.head()

,id,type,subtype,status,description,address,apn,applied,issued,final,demolished,new
0,2014-663052,SFD,NaN,FINALED,SFD,1650 MORTON AV,31818014.0,2014-07-18,2015-03-18,2017-10-27,1,NaN
1,2014-663372,SFD,NaN,FINALED,SFD,292 MARGARITA CT,16713051.0,2014-09-23,2015-05-19,2018-07-12,1,NaN
2,2014-663487,SFD,NaN,FINALED,SFD,1215 ALTAMEAD DR,19331041.0,2014-10-10,2015-02-02,2016-12-23,1,NaN
3,2014-663511,SFD,NaN,FINALED,New SFD,932 PARMA WY,18941065.0,2014-10-16,2015-04-27,2016-04-04,1,NaN
4,2014-663587,SFD,NaN,FINALED,New SFD,542 BENVENUE AV,18952062.0,2014-11-03,2015-03-16,2016-11-21,1,NaN


In [303]:
results['new'] = results['new'].fillna(1) 

In [304]:
results.head()

,id,type,subtype,status,description,address,apn,applied,issued,final,demolished,new
0,2014-663052,SFD,NaN,FINALED,SFD,1650 MORTON AV,31818014.0,2014-07-18,2015-03-18,2017-10-27,1,1.0
1,2014-663372,SFD,NaN,FINALED,SFD,292 MARGARITA CT,16713051.0,2014-09-23,2015-05-19,2018-07-12,1,1.0
2,2014-663487,SFD,NaN,FINALED,SFD,1215 ALTAMEAD DR,19331041.0,2014-10-10,2015-02-02,2016-12-23,1,1.0
3,2014-663511,SFD,NaN,FINALED,New SFD,932 PARMA WY,18941065.0,2014-10-16,2015-04-27,2016-04-04,1,1.0
4,2014-663587,SFD,NaN,FINALED,New SFD,542 BENVENUE AV,18952062.0,2014-11-03,2015-03-16,2016-11-21,1,1.0


In [305]:
results['totalunit'] = results['new'] - results['demolished']

In [317]:
results = results.query("not (totalunit == 0 and type in ['SFD', 'ADU'])")

In [325]:
results['permyear'] = results.issued.dt.year

In [326]:
results.to_csv('../data/clean_data/los_altos_permits.csv', index=False)